Expressing the pairwise gravitational potential energy between planets as a function of the planets' orbital elements is a common problem encountered in celestial mechanics. Usually, one wishes to find a series representation of the so-called 'disturbing function', ${\cal R}$, defined as

$$
-\frac{Gm_im_j}{|\pmb{r}_i-\pmb{r}_j|} + \frac{\tilde {\bf r}_i \cdot \tilde {\bf r}_j}{m_0} = -\frac{Gm_im_j}{a_j}{\cal R}^{(i,j)}~.
$$


The disturbing function is typically written as a cosine series in the planets' angular orbital elements. 
Defining $\pmb{\theta}_{i,j} = (\lambda_j,\lambda_i,\varpi_i,\varpi_j,\Omega_i,\Omega_j)$, we can write this cosine series as
$$
        {\cal R}^{(i,j)} = \sum_{\bf k}c_{\pmb{k}}(\alpha,e_i,e_j,I_i,I_j)\cos(\pmb{k}\cdot \pmb{\theta}_{i,j})
$$
where $\alpha = a_i/a_j$. 
Rotation and reflection symmetries of the planets' gravitational interactions dictate that $c_{\bf k}\ne 0$ only if $\sum_{l=1}^{6}k_l = 0$ and $k_5+k_6=2n$ where $n$ is an integer.

In classical disturbing function expansions, the cosine amplitudes $c_{\pmb{k}}$ are further expanded as Taylor series in powers of the planets' eccentricities $e$ and $s = \sin(I_i/2)$ as
$$
 c_{\bf k}(\alpha,e_i,e_j,I_i,I_j) = e_i^{|k_3|}e_j^{|k_4|}s_i^{|k_5|}s_j^{|k_6|}\sum_{\nu_1,\nu_2,\nu_3,\nu_4=0}^\infty {\tilde{C}}_{\bf k} ^{(\nu_1,\nu_2,\nu_3,\nu_4)}(\alpha)s_i^{2\nu_1}s_j^{2\nu_2}e_i^{2\nu_3}e_j^{2\nu_4}
$$

``celmech`` offers the capability to compute the individual disturbing function coefficients through the function `celmech.disturbing_function.DFCoeff_Ctilde`.

# The 5:2 resonance as an example 

In this short example, we'll look at the disturbing function term
$$
{\tilde C}^{(0,0,0,0)}_{(5,-2,0,-1,-2,0)}(\alpha) s_i^2e_{j}\cos(5\lambda_j - 2\lambda_i -2\Omega_i-\varpi_j)
$$
associated with a 5:2 MMR.

### The Murray & Dermott way

If you're used to doing this sort of thing with [Murray & Dermott's Solar System Dynamics textbook](https://ui.adsabs.harvard.edu/abs/2000ssd..book.....M/abstract), you would look in the appendix and, under Table B.12, find the cosine argument $j\lambda' + (3-j)\lambda - \varpi' - 2\Omega$ as entry 4D3.6.
This would tell you that the associated coefficient is $e's^2f_{87}$. Looking up  $f_{87}$ in Table B.15, you would find the expression
$$
\frac{1}{4}[2j\alpha + \alpha^2D]B_{j-2}
$$
Referring back to equation B.1, you would find that $B_{j} = b_{3/2}^{(j)}(\alpha)$ and that $D$ represents the differential operator $\frac{d}{d\alpha}$.


Combining this information and evaluating the above expression with $j=5$, you would arrive at 
$$
\begin{align}
{\tilde C}^{(0,0,0,0)}_{(5,-2,0,-1,-2,0)}(\alpha) &= \frac{1}{4}[(2\times 5)\alpha + \alpha^2D]B_{3}\\
&= \frac{10}{4}\alpha b_{3/2}^{(3)}(\alpha) + \frac{1}{4}\alpha^2\frac{d}{d\alpha}b_{3/2}^{(3)}(\alpha)
\end{align}
$$
and would then need to find a way to evaluated the Laplace coefficient $b_{3/2}^{(3)}(\alpha)$ and its derivative, finally arriving at a numerical value for the disturbing function coefficient.

### The `celmech` way

`celmech` automates all of the above steps! 


First we'll get an expression for our coefficient with `celmech.disturbing_function.df_coefficient_Ctilde`

In [1]:
from celmech.disturbing_function import df_coefficient_Ctilde

In [2]:
k = (5,-2,0,-1,-2,0)
nu = (0,0,0,0)
C = df_coefficient_Ctilde(*k,*nu)
print(C)

{(1, (1.5, 3, 0)): 2.5, (2, (1.5, 3, 1)): 0.25, ('indirect', 1): 0}


`celmech` represents combinations of Laplace coefficients and their derivatives as dictionaries.
In particular, the value of the coefficient is given by the sum over Laplace coefficients
contained in the dictionary entries: 
$$
    \sum C \times \alpha^p \frac{d^{n}}{d\alpha^{n}} b_{s}^{j}(\alpha)
$$
where the dictionary entries are in the form `{ (p,(s,j,n)) : C }` along with an entry `{'indirect' : C}` representing the value indirect terms.

In [3]:
# Print coefficient expression in LaTeX
print("C = ")
for key,val in C.items():
    if key[0]=='indirect':
        # indirect term
        print(r"+ {} /\sqrt{{\alpha}}".format(val))
    else:
        p,sjn = key
        s,j,n = sjn
        print(r"+{0:.4f} * \alpha^{1:d}\frac{{ d^{4:d} b_{2}^{{({3:d})}}(\alpha)  }}{{ d\alpha^{4:d} }} ".format(val,p,s,j,n))

C = 
+2.5000 * \alpha^1\frac{ d^0 b_1.5^{(3)}(\alpha)  }{ d\alpha^0 } 
+0.2500 * \alpha^2\frac{ d^1 b_1.5^{(3)}(\alpha)  }{ d\alpha^1 } 
+ 0 /\sqrt{\alpha}


We can use the function `evaluate_df_coefficient_dict` to compute the numerical value of the coefficient we're interested in:

In [4]:
from celmech.disturbing_function import evaluate_df_coefficient_dict

In [5]:
alpha = (2/5)**(2/3)
Cval = evaluate_df_coefficient_dict(C,alpha)
print("C = {}".format(Cval))

C = 2.4895211066855745
